## 알아둘 것

기본적인 기능은 `vanilla_python_notebook_v2.ipynb` 노트북에서 구현되었으며, `langgraph_notebook_v2.ipynb` 노트북은 이전 노트북에서 바닐라 파이썬으로 구현된 내용을 LangChain 기반으로 포팅하는 데 목적이 있습니다. 따라서, 전반적인 흐름을 설명하는 것은 생략하겠으나, LangChain에 특화된 추가적인 설명을 담고 있습니다.

> LangChain이 먼저 등장하고 LangGraph가 생겨났으며, LangGraph가 생기기 전 이미 LangChain 커뮤니티가 탄생했습니다. 또한 LangGraph는 LangChain과 밀접하게 연결되어 사용 가능하도록 디자인되었기 때문에, 이 노트북에서는 특정 라이브러리를 직접적으로 언급하지 않는 한 LangChain 이라는 용어를 두루 두루 사용합니다.

## Imports

LangChain 및 LangGraph 에서 제공하는 다양한 래퍼 클래스를 임포트하는 점을 눈여겨봅시다

In [ ]:
!pip install chromadb==1.3.7
!pip install langchain==1.2.0
!pip install langchain_chroma==1.1.0
!pip install langchain_community==0.4.1
!pip install langchain_upstage
!pip install langsmith

In [ ]:
!pip install transformers -U

In [1]:
import os
import uuid
import json
import requests
from typing import (
    TypedDict, Annotated, Any, List, Dict, Union, Optional, Sequence
)
from pydantic import BaseModel, Field, ConfigDict
from dotenv import load_dotenv

import chromadb
from transformers import AutoTokenizer

from langchain.tools import tool
from langchain_chroma import Chroma
from langchain_upstage import UpstageEmbeddings, ChatUpstage, UpstageDocumentParseLoader
from langchain.chat_models import init_chat_model
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_core.documents import Document
from langchain_core.callbacks import Callbacks
from langchain_core.language_models import BaseLanguageModel
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage, AIMessage, ToolMessage
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langgraph.graph.message import add_messages

from chromadb.errors import NotFoundError

from langchain_core.documents.compressor import BaseDocumentCompressor
from langchain_classic.retrievers.document_compressors import DocumentCompressorPipeline
from langchain_classic.retrievers.document_compressors.embeddings_filter import EmbeddingsFilter
from langchain_community.document_transformers.embeddings_redundant_filter import EmbeddingsRedundantFilter

from langsmith import Client, traceable
from langsmith.evaluation import evaluate
from langsmith.run_helpers import get_current_run_tree

## API 키 값 설정

In [35]:
# Colab
from google.colab import userdata

SERPER_API_KEY = userdata.get("SERPER_API_KEY")
UPSTAGE_API_KEY = userdata.get("UPSTAGE_API_KEY")
LANGSMITH_API_KEY = userdata.get("LANGSMITH_API_KEY")

os.environ["SERPER_API_KEY"] = SERPER_API_KEY
os.environ["UPSTAGE_API_KEY"] = UPSTAGE_API_KEY
os.environ["LANGSMITH_API_KEY"] = LANGSMITH_API_KEY

In [3]:
os.environ["LANGSMITH_API_KEY"] = LANGSMITH_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGSMITH_API_KEY

os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_ENDPOINT"]  = "https://api.smith.langchain.com"

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGCHAIN_TRACING_V2"] = "true"

os.environ["LANGSMITH_PROJECT"] = "PaperQA"
os.environ["LANGCHAIN_PROJECT"] = "PaperQA"

## 공통 변수 및 함수

앞서 `vanilla_python_notebook_v2.ipynb` 노트북에서 우리는 직접적으로 OpenAI SDK 및 `requests`와 같은 패키지를 통해 Upstage의 `solar-pro2`, `document-parse-250618`과 같은 모델과 상호 작용했으며, Chroma DB 엔진에서 Upstage의 `solar-embedding` 모델을 사용하기 위해서 `EmbeddingFunction[Documents]`를 상속받은 `UpstageEmbeddingFunction` 클래스를 정의했었습니다.

여전히 이 방식을 LangGraph에서도 사용할 수 있지만, LangGraph로 Agentic 워크플로를 구현할 때는 기본적으로 LangChain에서 래핑해 둔 클래스를 활용하는 것이 권장됩니다. 여기에는 다음 예시와 같은 이점이 있습니다.
- **트레이싱**: LangGraph는 그래프 기반 워크플로를 실행하는 전반적인 흐름을 "로그"로 남기고 모니터링할 수 있는 LangSmith와 간결한 통합을 지원합니다. 가령 LLM 호출시마다 발생하는 토큰량을 집계하고, 그래프의 각 부분 그래프나 노드에서 발생하는 Raw 입/출력의 기록이 그대로 남게 되는 것입니다. 이는 LangChain 기반으로 LLM 인터페이스, Tool 등과 같은 것들이 추상화되어 있기 때문에 가능합니다.

- **다양한 상호 운용 가능성**: 앞서 `solar-embedding` 모델을 Chroma DB에서 사용하기 위해서는 `UpstageEmbeddingFunction` 클래스를 정의했어야 했고, 이 클래스는 Chroma DB 엔진만을 위한 것이었습니다. 하지만, Vector DB를 지원하는 다양한 Open Source 솔루션과 Closed Source 서비스가 존재합니다. LangChain에서는 이들을 모두 추상화하고, 인터페이스의 결합도를 낮춰서 지원하는 모든 임베딩 함수를 지원하는 모든 Vector DB 엔진과 결합해서 사용할 수 있도록 해 줍니다.

이 노트북에서는 다음과 같은 래퍼를 활용합니다.

from langchain_upstage import UpstageEmbeddings, ChatUpstage, UpstageDocumentParseLoader

- `langchain_upstage.ChatUpstage`: Upstage의 `solar-pro2` 모델과 상호 작용하기 위한 래퍼입니다.
- `langchain_upstage.UpstageEmbeddings`: Upstage의 `solar-embedding` 모델과 상호 작용하기 위한 래퍼입니다.
- `langchain_upstage.UpstageDocumentParseLoader`: Upstage의 `document-parse-*` 모델과 상호 작용하기 위한 래퍼입니다.

`ChatUpstage` 클래스 인스턴스를 한 번만 생성해 두면, 이후 다양한 Agent에서 필요한 특정 도구들과 `bind_tools()` 메서드를 통해 바인딩된 특화 인스턴스를 쉽게 만들 수 있습니다. 또한, `reasoning_effort` 별로 인스턴스를 여러 개 준비해 두고 생각이 더 많이 필요한 작업을 다루는 Agent와 그렇지 않은 Agent 사이에 손쉬운 상호 운용이 가능하며, 보다 고성능 추론이 필요한 경우에는 OpenAI의 `GPT-5.2`와 같은 모델과도 손쉬운 상호 운용이 가능합니다.

In [4]:
PERSIST_DIR = "/tmp/chroma_db"

tokenizer = AutoTokenizer.from_pretrained("upstage/solar-pro-preview-instruct")

solar_chat = ChatUpstage(api_key=UPSTAGE_API_KEY, model="solar-pro2", reasoning_effort="high")
# solar_chat = init_chat_model('solar-pro2', reasoning_effort="high")
embedding_fn = UpstageEmbeddings(api_key=UPSTAGE_API_KEY, model="solar-embedding-1-large")

chroma_client = chromadb.PersistentClient(path=PERSIST_DIR)
ls_client = Client(
    api_url="https://api.smith.langchain.com", api_key=LANGSMITH_API_KEY
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
def clean_and_parse_json(text):
    import re
    try:
        match = re.search(r"```json\s*(.*?)\s*```", text, re.DOTALL)
        if match: text = match.group(1)
        else:
            match = re.search(r"(\{.*\})", text, re.DOTALL)
            if match: text = match.group(1)
        return json.loads(text)
    except:
        return None

In [6]:
def count_tokens(text: str) -> int:
    return len(tokenizer.encode(text))

# 2. 메시지 리스트를 받아서 토큰 제한에 맞춰 자르는 함수 (Context Management)
def truncate_tokens_if_needed(messages: list, max_tokens: int = 60000) -> list:
    current_tokens = 0
    truncated_messages = []

    system_msg = None
    if isinstance(messages[0], SystemMessage):
        system_msg = messages[0]
        current_tokens += count_tokens(system_msg.content)
        processing_messages = reversed(messages[1:]) # 시스템 제외 역순
    else:
        processing_messages = reversed(messages)

    # 2. 역순으로 담기 (최신 대화 우선)
    for msg in processing_messages:
        msg_tokens = count_tokens(str(msg.content)) # content가 없을 수도 있으므로 str변환 안전장치

        # ToolMessage는 내용이 길 수 있으므로 내부적으로 한 번 더 자름
        if isinstance(msg, ToolMessage):
            if msg_tokens > 20000: # 도구 결과가 너무 길면 강제 단축
                msg.content = msg.content[:20000] + "...(truncated)"
                msg_tokens = count_tokens(msg.content)

        if current_tokens + msg_tokens <= max_tokens:
            truncated_messages.append(msg)
            current_tokens += msg_tokens
        else:
            break

    final_messages = list(reversed(truncated_messages))
    while len(final_messages) > 0 and isinstance(final_messages[0], ToolMessage):
        final_messages.pop(0)

    if system_msg:
        final_messages.insert(0, system_msg)

    return final_messages

## InfoBuildAgent

`InfoBuildAgent`는 기본적으로 `vanilla_python_notebook_v2.ipynb` 노트북에서와 유사한 방식으로 구현되었습니다. 다만, `Serper.dev`, `Chroma DB`, 및 `Document Parse` 제품 사용 방법이 LangChain 커뮤니티에서 제공하는 래퍼 클래스로 대체되었습니다. 더불어 LangChain에서 제공하는 `RecursiveCharacterTextSplitter` 같은 유용한 유틸리티 기능을 활용해서, 큰 덩어리 텍스트를 유의미한 청크로 손쉽게 자르는 방법을 보여줍니다.

> 텍스트를 분할하는 방법은 다양할 수 있으며, 어디서 부터 어디까지 잘랐느냐에 따라서 비슷한 텍스트 덩어리 처럼 보이더라도 임베딩 벡터 공간에서의 값이 상이할 수 있습니다. 따라서 텍스트를 최대한 다양한 방법으로 잘라서, 다양한 방식으로 저장해 두는 편이 좋습니다. 이를 위해 `RecursiveCharacterTextSplitter` 클래스가 허용하는 파라미터 값들을 숙지해 두는 것이 좋습니다 [[링크](https://reference.langchain.com/python/langchain_text_splitters/?h=recursivecharactertextsplitter#langchain_text_splitters.TextSplitter.__init__)].

LangChain 에서는 도구를 정의하는 두 가지 방법을 지원합니다.
1. `@tool` 데커레이터: 파이썬 함수 위에 `@tool` 데커레이터를 사용하여 해당 함수를 LangChain Tool 객체로 변환합니다. 코드가 간결하며, 기존 함수를 빠르게 도구로 만들 수 있습니다. 닥스트링이 도구의 설명으로 사용됩니다 (이는 에이전트가 도구를 사용해야 할지 결정하는 데 중요합니다).

2. `Tool`: `langchain.tools` 모듈의 `Tool` 클래스로 직접 도구를 정의합니다. 이 방식은 도구의 이름, 설명, 실행 함수를 명시적으로 전달해야해서, 도구의 이름과 설명을 실행 함수와 분리하여 명시적으로 제어할 수 있다는 장점이 있습니다.

이 튜토리얼에서는 `@tool` 데커레이터를 이용한 방법만을 사용하지만, `langchain.tools.Tool`을 사용하는 방법과 언제 `@tool` 대신 사용하는 것이 좋은지에 대한 내용을 살펴보면 좋습니다 [[링크](https://docs.langchain.com/oss/python/langchain/tools)].

> 여기서 눈여겨보면 좋을 만한 포인트는 `vanilla_python_notebook_v2.ipynb`에서 이미 다양한 실험으로 검증된 프롬프트가 그대로 재활용될 수 있는 부분이다. Agentic Workflow를 구현하는 코드량은 사실상 많지 않으며, 두뇌 역할을 하는 LLM이 잘 알아듣는지 그러지 못하는지를 확인해 가며 프롬프트와 컨텍스트를 최적화해 나가는 디버깅 시간이 구현 시간에 비해 압도적으로 많이 드는 편이다.

> 즉시 LangGraph로 구현을 시작해도 좋지만, 수 많은 래퍼 클래스와 함수를 사용함에 따라 제어 가능/불가능한 영역이 분명히 존재하며, 경우에 따라서는 비용 제어도 쉽지 않을 수 있다. 따라서 Vanilla 파이썬으로 초안을 잡은 뒤 LangGraph로 포팅하는 것도 좋은 작업 흐름이라고 볼 수 있다 (특히 탄탄하게 Vanilla 파이썬으로 구현된 경우, 바이브 코딩으로 매우 손쉬운 포팅이 가능하다).

### Tools 정의

#### `search_arxiv`

In [12]:
@tool
def search_arxiv(search_query: str) -> str:
    """
    Search arXiv for a paper based on keywords.
    search_query should be appropriate for arXiv search.
    Returns the title and arXiv ID of the most relevant paper.
    """
    search = GoogleSerperAPIWrapper()
    results = search.results(search_query)
    for result in results['organic']:
        if 'arxiv' in result['link'].lower():
            title = result['title']
            paper_id = result['link'].split('/')[-1]

            return f"Found Paper - Title: {title}, ID: {paper_id}"

    return "Paper not found. Try with different search query."

#### `check_vector_db_cache`

In [13]:
@tool
def check_vector_db_cache(arxiv_id: str) -> str:
    """
    Check if the paper is already indexed in the VectorDB.
    Returns 'True' if cached, 'False' otherwise.
    """
    try:
        col = chroma_client.get_collection(name=arxiv_id)
        hit = (col.count() > 0)
        return f"Cache Hit: True"
    except NotFoundError:
        return "Cache Hit: False"
    except Exception:
        return "Cache Hit: False"

#### `download_pdf`

In [14]:
@tool
def download_pdf(arxiv_id: str):
    """
    Download the PDF file from arXiv using the ID.
    Returns the file path of the downloaded PDF.
    """
    pdf_url = f"https://arxiv.org/pdf/{arxiv_id}.pdf"
    save_path = f"./{arxiv_id}.pdf"
    try:
        response = requests.get(pdf_url)
        with open(save_path, "wb") as f:
            f.write(response.content)
        return f"PDF file is saved at '{save_path}'"
    except Exception as e:
        return f"Download Error: {e}"

#### `parse_and_store_to_vectordb`

In [15]:
@tool
def parse_and_store_to_vectordb(arxiv_id: str, file_path: str) -> str:
    """
    Parse the PDF file into Markdown text using Upstage Document Parse API,
    split it into chunks, and store them in VectorDB.
    """
    try:
        # page by page parsing: split="page"
        loader = UpstageDocumentParseLoader(file_path, api_key=UPSTAGE_API_KEY, split="none", output_format="markdown")
        parsed_docs = loader.load()  # Returning List[Document]

        # chunking
        splitter = RecursiveCharacterTextSplitter(
            chunk_size=2000,
            # chunk_overlap=200,
        )
        chunked_docs = splitter.split_documents(parsed_docs)
        for d in chunked_docs:
            d.metadata = {}   # 전부 제거

        # VectorStore: LangChain Chroma (없으면 생성/있으면 재사용)
        vectorstore = Chroma(
            collection_name=arxiv_id,
            embedding_function=embedding_fn,
            persist_directory=PERSIST_DIR,
        )

        ids = [f"{arxiv_id}_{i}" for i in range(len(chunked_docs))]
        vectorstore.add_documents(chunked_docs, ids=ids)

        return f"Paper {arxiv_id} store: True. Status: Success ({len(chunked_docs)} chunks)"

    except Exception as e:
        return f"Store Error: {e}"


### InfoBuildAgent 정의

In [16]:
instruction_info_build="""You are the 'InfoBuildAgent', responsible for identifying target papers and building a knowledge base.

# Goal
Your goal is to ensure that the paper requested by the user is stored in the VectorDB (Cache).
You must output the final arXiv ID and status when the job is done.

# Workflow (Strictly Follow This Order)
1. **Identify arXiv ID**:
   - Check if the user provided an arXiv ID (e.g., '2310.12345').
   - If NOT, use `search_arxiv(search_query)` tool to find the ID from the internet. You should generate appropriate search query for arXiv ID of the paper.

2. **Check Cache**:
   - Once you have the arXiv ID, use `check_vector_db_cache` to see if it's already stored.
   - If it returns "True", STOP and report the ID.

3. **Build Knowledge (If Cache Miss)**:
   - If cache is "False", you must proceed with the following steps in order:
     A. `download_pdf(arxiv_id)`
     B. `parse_and_store_to_vectordb(arxiv_id, file_path)`

4. **Final Answer**:
   - When the paper is confirmed to be in the DB (either via cache hit or after storing),
   - Respond the arXiv ID and status (success or fail) in JSON format (```json```) without any style. Do not respond anything else.
"""

LangGraph에서는 복잡한 워크플로우와 다중 에이전트 상호작용을 효율적으로 관리하기 위해 `State(상태)`라는 개념을 도입했습니다. `State`는 에이전트 간 데이터 전달, 작업 진행 상황 추적, 컨텍스트 유지 등을 위한 중앙 집중식 데이터 구조로, 각 노드가 공유하고 업데이트할 수 있는 공통 인터페이스를 제공합니다.

그리고 몇 가지 편리한 기능이 함께 제공됩니다.
- **Annotated**는 상태 필드에 메타데이터(예: 설명, 타입 힌트, 검증 규칙)를 추가하여 코드의 가독성과 유지보수성을 높이는 데 사용됩니다. Python의 타입 힌트와 유사하게 동작하며, 상태 관리 로직을 더 명확하고 안전하게 만들어줍니다.

- **add_messages**는 대화형 애플리케이션에서 메시지 히스토리를 상태에 자동으로 누적시키는 유틸리티 함수로, 에이전트 간 대화 기록을 관리하고 컨텍스트를 유지하는 데 유용합니다. 가령 사용자 입력이나 에이전트 응답을 상태에 자동으로 추가하여 워크플로우의 연속성을 보장합니다.

아래의 `InfoBuildAgentState` 처럼 `messages`를 `Annotated[List[BaseMessage], add_messages]`라고 정의내리면, 기본적으로 `messages`는 `BaseMessage`를 상속받은 메시지 유형 클래스의 인스턴스만을 담아내는 그릇이 되며, LangGraph 노드가 `{"messages": [추가_메시지]}` 같이 값을 반환하는 경우, 이 반환된 값이 자동으로 `messages`에 누적되는 (숨은) 기능이 작동하게 됩니다.



In [17]:
class InfoBuildAgentState(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]

info_build_tools = [
   search_arxiv,
   check_vector_db_cache,
   download_pdf,
   parse_and_store_to_vectordb
]

LangChain에서 제공하는 채팅 인터페이스를 상속받은 모든 클래스는 `bind_tools` 메서드를 통해, 밑바탕에 깔린 LLM과 도구를 결합한 새로운 인스턴스를 반환합니다. 즉, Agent 마다 목적에 맞는 서로 다른 도구를 바인딩하게 되는 데, 이들을 일일이 매번 생성할 필요 없이, 1차적으로 채팅 인터페이스에 대한 인스턴스를 생성한 뒤, `bind_tools` 메서드를 사용하여 특화 인스턴스를 생성할 수 있습니다.

In [18]:
llm_info_build = solar_chat.bind_tools(info_build_tools)

In [19]:
def info_build_agent(state: InfoBuildAgentState):
    messages = state["messages"]

    if not isinstance(messages[0], SystemMessage):
        messages = [
            SystemMessage(content=instruction_info_build)
        ] + messages

    messages = truncate_tokens_if_needed(messages, max_tokens=60000)
    response = llm_info_build.invoke(messages)

    return {"messages": [response]}

info_build_tool_node = ToolNode(info_build_tools)

# --- 그래프 구성 (Workflow) ---
info_build_workflow = StateGraph(InfoBuildAgentState)

info_build_workflow.add_node("info_build_agent", info_build_agent)
info_build_workflow.add_node("info_build_tools", info_build_tool_node)

info_build_workflow.set_entry_point("info_build_agent") # 시작점

def should_continue(state: InfoBuildAgentState):
    last_message = state["messages"][-1]
    # LLM이 도구 호출(tool_calls)을 포함했는지 확인
    if last_message.tool_calls:
        return "tools"
    return END

info_build_workflow.add_conditional_edges(
    "info_build_agent",
    should_continue,
    {
        "tools": "info_build_tools",
        END: END
    }
)

info_build_workflow.add_edge("info_build_tools", "info_build_agent")
info_build_graph = info_build_workflow.compile()

### InfoBuildAgent 평가

#### 평가 데이터셋 정의

In [19]:
test_dataset = [
    ("Attention is All You Need 논문 찾아줘", "1706.03762"),
    ("Llama 3 논문 (The Llama 3 Herd of Models) 처리해줘", "2407.21783"),
    ("BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding", "1810.04805"),
    ("Chain-of-Thought Prompting Elicits Reasoning in Large Language Models 논문", "2201.11903"),
    ("LoRA: Low-Rank Adaptation of Large Language Models", "2106.09685")
]

#### LangSmith 를 통해 평가 및 확인

In [20]:
import uuid

dataset_name = "PaperQA_BuildInfoAgent_Eval_DS"

# 데이터셋이 없는 경우에만 새로 생성
if not ls_client.has_dataset(dataset_name=dataset_name):
    dataset = ls_client.create_dataset(dataset_name=dataset_name)
    for query, arxiv_id in test_dataset:
        ls_client.create_example(
            inputs={"query": query},
            outputs={"expected_arxiv_id": arxiv_id},
            dataset_id=dataset.id,
        )

# 2. 평가를 위한 타겟 함수 (그래프 실행)
def predict_info_build(inputs: dict):
    # 그래프 실행
    config = {"configurable": {"thread_id": str(uuid.uuid4())}, "recursion_limit": 100} # 실제 세션은 LangSmith가 분리해줌
    result = info_build_graph.invoke({"messages": [("user", inputs["query"])]}, config=config)

    # 최종 답변 추출
    last_msg = result["messages"][-1].content
    parsed = clean_and_parse_json(last_msg)

    return {
        "actual_arxiv_id": parsed.get("arxiv_id", "N/A") if parsed else "ERROR",
        "full_response": last_msg
    }

# 3. 커스텀 평가 지표 (사용자 정의 함수 활용)
def arxiv_id_match_evaluator(run, example):
    prediction = run.outputs.get("actual_arxiv_id")
    reference = example.outputs.get("expected_arxiv_id")

    score = 1 if prediction == reference else 0
    return {"key": "arxiv_id_match", "score": score}

# 4. 평가 실행
results = evaluate(
    predict_info_build,
    data=dataset_name,
    evaluators=[arxiv_id_match_evaluator],
    experiment_prefix="PaperQA-InfoBuildAgent-Eval",
    num_repetitions=3,
    client=ls_client,
)

print(f"\n✅ 평가 완료! LangSmith에서 상세 로그를 확인하세요: {results.experiment_name}")

View the evaluation results for experiment: 'PaperQA-InfoBuildAgent-Eval-b962ec37' at:
https://smith.langchain.com/o/06a2d763-c3d4-5ed5-af24-a483205bb09c/datasets/fb0b93c8-3a32-4d3c-bb20-c123322b2944/compare?selectedSessions=7b30bd7c-c2e0-4660-a0b2-17c2f8499a02




0it [00:00, ?it/s]


✅ 평가 완료! LangSmith에서 상세 로그를 확인하세요: PaperQA-InfoBuildAgent-Eval-b962ec37


## InfoExtractAgent

`InfoExtractAgent`의 역할은 `vanilla_python_notebook_v2.ipynb`에서와 달라지지 않았지만, 구현 방식은 크게 다릅니다. 즉, 여전히 Chroma DB로 부터 다섯 개의 쿼리에 대해 각각 10개씩 데이터를 인출한 다음, **(1)** 중복을 제거하고, **(2)** LLM-as-a-Judge 방식으로 점수를 매겨 리랭크하는 설명에는 차이가 없지만, **(1)** 과 **(2)** 를 수행하는 방식을 LangChain의 `Compressor` 기반 아키텍처를 활용하여 더 체계적이고 확장 가능한 방식으로 이를 구현했습니다.

### Tools

#### `search_and_rerank`

`LLMReranker`는 LangChain의 `BaseDocumentCompressor`를 상속받아 문서 압축(리랭킹) 기능을 표준화된 인터페이스에 맞춰 구현했습니다. `BaseDocumentCompressor`는 문서 집합을 입력받아 필터링 또는 재정렬하는 추상 클래스로, LangChain의 문서 처리 파이프라인과 자연스럽게 통합될 수 있습니다. 이를 통해 기존의 리트리버나 체인 컴포넌트와 호환성을 유지하면서, 커스텀 리랭킹 로직을 플러그인 형태로 삽입할 수 있습니다.

특히, `compress_documents` 메서드는 LangChain의 문서 처리 흐름에서 자동으로 호출되므로, 이 인터페이스를 준수하면 별도의 어댑터 없이도 체인 내 다른 컴포넌트와 원활히 연동됩니다. 또한, `BaseDocumentCompressor`를 사용하면 중복 제거, 리랭킹, 필터링 등 다양한 압축 전략을 일관된 방식으로 적용할 수 있어 코드의 확장성과 유지보수성을 높일 수 있습니다.

`LLMReranker`는 LLM(Upstage의 `solar-pro2`)에게 사용자 입력(질문)과 문서 목록을 제공하고, 둘 사이의 연관성 점수를 JSON 형식으로 출력하기를 요청함으로써 **문맥적 관련성 평가**를 수행합니다. 메서드 내부에서는 문서를 번호화하여 프롬프트로 변환하고, LLM의 응답을 파싱해 `min_score` 이상인 문서만 선별하여 점수 순으로 재정렬합니다. 이는 정적 필터링이 아닌 **동적 평가**를 위함입니다.

In [20]:
class ScoredDoc(BaseModel):
    id: int = Field(description="Document index")
    score: int = Field(description="Relevance score (0-10)")

class LLMReranker(BaseDocumentCompressor):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    llm: BaseLanguageModel = Field(exclude=True)
    min_score: int = 7

    def compress_documents(
        self,
        documents: Sequence[Document],
        query: str,
        callbacks: Optional[Callbacks] = None,
    ) -> List[Document]:
        numbered_docs = "\n\n".join(
            [f"[ID: {i}] {doc.page_content[:300]}..." for i, doc in enumerate(documents)]
        )

        prompt = f"""
Queries: {query}
Candidates:
{numbered_docs}

Task:
1. Score each candidate (0-10) based on relevance to the queries.
2. Filter out anything below {self.min_score}.
3. Output strictly in JSON format: [{{"id": 0, "score": 9}}, ...]
"""

        resp = self.llm.invoke(prompt)
        text = getattr(resp, "content", resp)

        try:
            scored_list = json.loads(text)
            scored_list = [x for x in scored_list if x["score"] >= self.min_score]
            scored_list.sort(key=lambda x: x["score"], reverse=True)
            return [documents[item["id"]] for item in scored_list]
        except Exception:
            return list(documents)

`search_and_rerank` 도구는 `DocumentCompressorPipeline`을 기반으로 문서 압축 파이프라인을 구성하여, 검색 결과에 대해 중복 제거 및 리랭킹을 순차적으로 적용합니다. 첫 단계인 중복 제거는 `EmbeddingsRedundantFilter`로 임베딩 유사도를 기준으로 작동하며(`similarity_threshold=0.9`), 이는 Chroma DB로부터 검색된 문서들 사이에서 의미적으로 중복된 콘텐츠를 제거하여 후속 리랭킹의 효율성을 높입니다. 이후 `LLMReranker`가 LLM 기반 관련성 평가로 최적의 문서를 선별함으로써, 정확도와 다양성을 동시에 확보하는 구조입니다. 파이프라인 패턴을 채택함으로써 각 단계의 로직을 모듈화하고, 필요 시 추가 변환기를 유연하게 삽입할 수 있습니다.

In [21]:
@tool
def search_and_rerank(queries: Union[str, List[str]], arxiv_id: str, k: int = 5, top_k: int = 10):
    """
    Search with multiple queries -> Deduplicate -> Rerank -> Return Top-K chunks.
    queries: List of search query strings.
    arxiv_id: The target paper's ID.
    """
    # 입력 안전장치 (문자열로 들어온 경우 리스트로 변환)
    if isinstance(queries, str):
        try:
            queries = json.loads(queries)
        except:
            queries = [queries]

    retriever = Chroma(
        collection_name=arxiv_id,
        embedding_function=embedding_fn,
        client=chroma_client,
    ).as_retriever(search_kwargs={"k": k})

    # 문서 압축 파이프라인 (중복 제거 + 재랭킹)
    pipeline = DocumentCompressorPipeline(
        transformers=[
            EmbeddingsRedundantFilter(embeddings=embedding_fn, similarity_threshold=0.9),
            LLMReranker(llm=solar_chat),
        ]
    )

    # 검색 및 재랭킹
    try:
        all_docs = []
        for query in queries:
            docs = retriever.invoke(query)
            all_docs.extend(docs)

        # 파이프라인 적용
        compressed_docs = pipeline.compress_documents(
            documents=all_docs,
            query=queries[0],  # 또는 쿼리 병합 로직 추가
        )

        return json.dumps([doc.page_content for doc in compressed_docs[:top_k]])

    except Exception as e:
        return f"Error: {str(e)}"

### InfoExtractAgent 정의

In [22]:
instruction_info_extract ="""
You are the 'InfoExtractAgent'. Your goal is to retrieve roughly **10 high-quality information chunks** in a SINGLE attempt.

# Workflow (One-Shot Batch Processing)
1. **Check Status**: IF input says "fail", STOP and output `{"status": "fail"}`.
2. **Generate Queries**: Analyze the user query and generate **5 DISTINCT search queries** (Python List of strings).
3. **Search & Rerank**: Call `search_and_rerank(queries=[...], arxiv_id=..., k=5, top_k=10)`.
4. **Final Answer**:
   Return strictly JSON format: `{"status": "success", "arxiv_id": "...", "chunks": ["chunk1", "chunk2", ...]}`
   Do NOT output anything else.
"""

In [23]:
class InfoExtractAgentState(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]

info_extract_tools = [search_and_rerank]
llm_info_extract = solar_chat.bind_tools(info_extract_tools)

In [24]:
def info_extract_agent(state: InfoExtractAgentState):
    messages = state["messages"]

    if not isinstance(messages[0], SystemMessage):
        messages = [
            SystemMessage(content=instruction_info_extract)
        ] + messages

    messages = truncate_tokens_if_needed(messages, max_tokens=60000)

    response = llm_info_extract.invoke(messages)
    return {"messages": [response]}

info_extract_tool_node = ToolNode(info_extract_tools)

info_extract_workflow = StateGraph(InfoExtractAgentState)
info_extract_workflow.add_node("info_extract_agent", info_extract_agent)
info_extract_workflow.add_node("info_extract_tools", info_extract_tool_node)

info_extract_workflow.set_entry_point("info_extract_agent")

def should_continue(state: InfoExtractAgentState):
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "tools"
    return END

info_extract_workflow.add_conditional_edges(
    "info_extract_agent",
    should_continue,
    {
        "tools": "info_extract_tools",
        END: END
    }
)

info_extract_workflow.add_edge("info_extract_tools", "info_extract_agent")
info_extract_graph = info_extract_workflow.compile()

### InfoExtractAgent 평가

#### 평가 데이터셋 정의

In [26]:
raw_dataset = [
    (
        "Llama 3의 모델 아키텍처 특징은 무엇인가?",
        "2407.21783",
        ["Transformer", "GQA", "attention", "layers"]
    ),
    (
        "Llama 3의 학습 데이터 규모는 얼마나 되나?",
        "2407.21783",
        ["15T", "tokens", "multilingual", "pre-training"]
    ),
    (
        "Llama 3의 벤치마크 성능 결과는?",
        "2407.21783",
        ["MMLU", "GSM8K", "HumanEval", "GPT-4"]
    ),
    (
        "Llama 3의 안전성(Safety) 확보 방법은?",
        "2407.21783",
        ["Llama Guard", "CyberSecEval", "red teaming", "safety"]
    ),
    (
        "Llama 3 405B 모델의 하드웨어 인프라는?",
        "2407.21783",
        ["H100", "GPU", "cluster", "RoCE"]
    ),
]

In [27]:
dataset_name = "PaperQA_InfoExtractAgent_Eval_DS"

if not ls_client.has_dataset(dataset_name=dataset_name):
    dataset = ls_client.create_dataset(dataset_name=dataset_name, description="ArXiv Chunk Retrieval Keyword Test")
    for query, arxiv_id, keywords in raw_dataset:
        ls_client.create_example(
            inputs={"query": query, "arxiv_id": arxiv_id},
            outputs={"expected_keywords": keywords},
            dataset_id=dataset.id,
        )
    print(f"✅ Dataset created: {dataset_name}")
else:
    print(f"ℹ️ Dataset already exists: {dataset_name}")

def predict_info_extract(inputs: Dict[str, Any]) -> Dict[str, Any]:
    user_query = inputs["query"]
    target_id = inputs["arxiv_id"]

    prev_result_mock = {"status": "success", "arxiv_id": target_id}

    context_message = (
        "Here is the context from the previous step:\n"
        f'1. **Original User Query**: "{user_query}"\n'
        f"2. **Previous Step Result**: {json.dumps(prev_result_mock, ensure_ascii=False)}\n\n"
        "Proceed with the workflow based on the 'status' field."
    )

    config = {"configurable": {"thread_id": str(uuid.uuid4())}, "recursion_limit": 100}
    result = info_extract_graph.invoke({"messages": [("user", context_message)]}, config=config)

    last_msg = result["messages"][-1].content
    parsed = clean_and_parse_json(last_msg)

    return {
        "status": parsed.get("status", "ERROR") if isinstance(parsed, dict) else "ERROR",
        "arxiv_id": parsed.get("arxiv_id", "N/A") if isinstance(parsed, dict) else "N/A",
        "chunks": parsed.get("chunks", []) if isinstance(parsed, dict) else [],
        "full_response": last_msg,
    }


def keyword_exact_score_evaluator(run, example):
    outputs = run.outputs or {}
    chunks: List[str] = outputs.get("chunks") or []
    expected_keywords: List[str] = example.outputs["expected_keywords"]

    if not chunks or not expected_keywords:
        return {"key": "keyword_score_100", "score": 0, "comment": "no chunks or no keywords"}

    full_text = " ".join(chunks).lower()

    per_kw = 100.0 / len(expected_keywords)
    hit = 0
    missing = []

    for kw in expected_keywords:
        if kw.lower() in full_text:
            hit += 1
        else:
            missing.append(kw)

    score = per_kw * hit  # 0~100
    comment = f"hit {hit}/{len(expected_keywords)}; missing={missing}" if missing else f"hit {hit}/{len(expected_keywords)}"

    return {"key": "keyword_score_100", "score": score, "comment": comment}

results = evaluate(
    predict_info_extract,
    data=dataset_name,
    evaluators=[keyword_exact_score_evaluator],
    experiment_prefix="PaperQA-InfoExtractAgent-Keyword100",
    num_repetitions=3,
    client=ls_client,
)

print(f"\n✅ 평가 완료! LangSmith에서 상세 로그 확인: {results.experiment_name}")

ℹ️ Dataset already exists: PaperQA_InfoExtractAgent_Eval_DS
View the evaluation results for experiment: 'PaperQA-InfoExtractAgent-Keyword100-63220b6e' at:
https://smith.langchain.com/o/06a2d763-c3d4-5ed5-af24-a483205bb09c/datasets/d74c476b-712b-4273-9978-24bd4d76ab5a/compare?selectedSessions=0275f8f2-5979-414f-a3e5-a8b79cb64b1c




0it [00:00, ?it/s]


✅ 평가 완료! LangSmith에서 상세 로그 확인: PaperQA-InfoExtractAgent-Keyword100-63220b6e


## AnswerGenAgent

In [25]:
class AnswerGenAgentState(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]

instruction_answer_gen = """
You are an expert academic research assistant.
Your goal is to synthesize the provided context chunks into a clear, coherent, and accurate answer.

**Guidelines:**
1. **Tone**: Professional and objective.
2. **Language**: Korean.
3. **Constraint**: Do NOT hallucinate. Answer based **ONLY** on the provided context.
4. **Citation**: Cite the source index (e.g., [Source 1]) when referencing specific information.
"""

In [26]:
def answer_gen_agent(state: AnswerGenAgentState):
    messages = state["messages"]

    if not isinstance(messages[0], SystemMessage):
        messages = [SystemMessage(content=instruction_answer_gen)] + messages

    messages = truncate_tokens_if_needed(messages, max_tokens=60000)

    response = solar_chat.invoke(messages)
    return {"messages": [response]}

answer_gen_workflow = StateGraph(AnswerGenAgentState)
answer_gen_workflow.add_node("answer_gen_agent", answer_gen_agent)
answer_gen_workflow.set_entry_point("answer_gen_agent")
answer_gen_workflow.add_edge("answer_gen_agent", END)
answer_gen_graph = answer_gen_workflow.compile()

## EvaluateAgent

In [27]:
# 1. State 정의
class EvaluateAgentState(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]

# 2. Instruction 정의
instruction_eval_agent = """
You are a **Critical QA Auditor** for a RAG system.
Your goal is to evaluate the generated answer with **extreme strictness**.

# Scoring Rubric (Score 1-10)
1. **Faithfulness (50%)**:
   - Penalize hallucinations (info not in context).
   - Penalize missing or wrong citations.
   - Score 10: Fully supported by context.

2. **Relevance (30%)**:
   - Does it answer the specific question?
   - Score 10: Direct and precise answer.

3. **Style (20%)**:
   - Is it in Korean? Is the format clean?

# Output Format
Return ONLY a strictly valid JSON object:
{
  "faithfulness": { "score": int, "reason": "..." },
  "relevance": { "score": int, "reason": "..." },
  "style": { "score": int, "reason": "..." },
  "final_score": float
}
"""

In [28]:
@traceable(name="EvaluateAgent_Run") # LangSmith 트레이스 이름 지정
def evaluate_agent(state: EvaluateAgentState):
    messages = state["messages"]

    if not isinstance(messages[0], SystemMessage):
        messages = [SystemMessage(content=instruction_eval_agent)] + messages

    messages = truncate_tokens_if_needed(messages, max_tokens=60000)
    response = solar_chat.invoke(messages)

    try:
        eval_json = clean_and_parse_json(response.content)

        if eval_json:
            rt = get_current_run_tree()

            if rt:
                run_id = rt.trace_id # rt.id
                metrics = ["faithfulness", "relevance", "style"]

                for key in metrics:
                    if key in eval_json:
                        score_val = eval_json[key]["score"]
                        # normalized_score = score_val / 10.0

                        ls_client.create_feedback(
                            run_id=run_id,
                            key=key,
                            score=score_val,
                            comment=eval_json[key]["reason"]
                        )

                if "final_score" in eval_json:
                    ls_client.create_feedback(
                        run_id=run_id,
                        key="final_score",
                        score=eval_json["final_score"]
                    )

                print(f"   ✅ [LangSmith] Feedback 전송 완료 (Run ID: {run_id})")
            else:
                print("   ⚠️ [LangSmith] Run Tree를 찾을 수 없습니다.")
        else:
            print("   ⚠️ [EvaluateAgent] JSON 파싱 실패로 Feedback 전송 건너뜀")

    except Exception as e:
        print(f"   ❌ [LangSmith] Feedback 전송 중 에러 발생: {e}")
    # ----------------------------------------------

    return {"messages": [response]}

evaluate_workflow = StateGraph(EvaluateAgentState)
evaluate_workflow.add_node("evaluate_agent", evaluate_agent)
evaluate_workflow.set_entry_point("evaluate_agent")
evaluate_workflow.add_edge("evaluate_agent", END)

evaluate_graph = evaluate_workflow.compile()

## Super Graph

우리는 `InfoBuildAgent`, `InfoExtractAgent`, `AnswerGenAgent` 세 개의 Agent를 정의했습니다. 각각의 Agent는 고유 목적을 달성하기 위해 정의된 것으로, 각자 내부에서 일어나는 일을 서로 알 필요는 없습니다. 단지, 각 Agent가 끝에 도출한 결과 값을 넘겨 받아 작업을 이어나가면 되는 것입니다. 이를 위해 각 Agent별로 메시지 히스토리를 관리하는 편이 효율적입니다.

물론, 모든 Agent를 연결한 전체 그래프 차원에서 메시지 히스토리를 관리할 수도 있지만, 이 경우 컨텍스트의 길이가 굉장히 비대해 지는 결과를 초래하며, 결국에는 LLM의 판단을 흐리게 만듭니다. 특히 소규모 모델, 그리고 Context 길이를 길게 지원하지 않으면서 Context 에서 데이터를 인출하는 능력이 떨어지는 모델을 활용할 때에는 이러한 불안 요소를 제거해야만 합니다. 이는 오늘날 최고의 모델인 `Gemini 3.0 Pro` 및 `GPT-5.2` 조차도 어려워하는 문제로, 이들보다 훨씬 소규모 모델인 `solar-pro2`를 활용할 때에는 반드시 필요한 작업입니다.

각 에이전트가 집중해야 할 "맥락"을 관리한다는 차원에서, 이를 일종의 컨텍스트 엔지니어링으로 볼 수 있습니다. 이를 위해서, 우리는 그래프 전체를 관통하는 `State(상태)` 정보를 아래처럼 각 에이전트별로 대화 히스토리를 격리하는 형태로 정의해볼 수 있습니다.
- `user_query`: 최초 사용자가 입력한 질문
- `build_logs`: `InfoBuildAgent` 에이전트에서 쌓이는 대화 히스토리
- `extract_logs`: `InfoExtractAgent` 에이전트에서 쌓이는 대화 히스토리
- `answer_logs`: `AnswerGenAgent` 에이전트에서 쌓이는 대화 히스토리
- `process_status`: 각 단계별 작업 완료/실패 여부

In [29]:
class MainState(TypedDict):
    user_query: str

    build_logs: Annotated[List[BaseMessage], add_messages]
    extract_logs: Annotated[List[BaseMessage], add_messages]
    answer_logs: Annotated[List[BaseMessage], add_messages]
    eval_logs: Annotated[List[BaseMessage], add_messages] # +

    process_status: str

각 Agent를 호출하고, 호출 결과에 따라 `process_status` 값을 업데이트하는 함수 세 개를 정의합니다. 결국 각 함수는 전체 그래프를 구성하는 노드로 편입되고, 각 노드의 결과 상태 값에 따라 다음 노드로 값을 전달하는 `check_*` 함수 두 개를 정의합니다.

In [30]:
def call_info_builder(state: MainState):
    print("\n [Main] InfoBuildAgent 호출 ...")

    fresh_messages = [HumanMessage(content=state["user_query"])]
    sub_inputs = {"messages": fresh_messages}
    sub_result = info_build_graph.invoke(sub_inputs)

    result_messages = sub_result["messages"]
    last_message = result_messages[-1]

    return {
        "build_logs": result_messages,
        "process_status": "done"
    }

def call_info_extractor(state: MainState):
    print("\n [Main] InfoExtractAgent 호출 ...")

    last_build_msg = state["build_logs"][-1]
    parsed = clean_and_parse_json(last_build_msg.content)

    handoff_msg = (
        f"Here is the context from the previous step:\n"
        f"1. **Original User Query**: \"{state['user_query']}\"\n"
        f"2. **Previous Step Result**: {last_build_msg.content}\n\n"
        f"Proceed with the workflow based on the 'status' field."
    )

    sub_inputs = {"messages": [HumanMessage(content=handoff_msg)]}
    sub_result = info_extract_graph.invoke(sub_inputs)

    return {
        "extract_logs": sub_result["messages"],
        "process_status": "success"
    }

def call_answer_gen(state: MainState):
    print("\n [Main] AnswerGenAgent 호출 ...")

    if not state.get("extract_logs"):
        return {
            "answer_logs": [AIMessage(content="죄송합니다. 정보를 추출하는 데 실패하여 답변을 생성할 수 없습니다.")],
            "process_status": "fail"
        }

    last_extract_msg = state["extract_logs"][-1]
    parsed_result = clean_and_parse_json(last_extract_msg.content)

    chunks = parsed_result.get("chunks", [])
    if not chunks:
        return {
            "answer_logs": [AIMessage(content="검색된 문서에서 유의미한 정보를 찾지 못했습니다.")],
            "process_status": "fail"
        }

    formatted_context = "\n\n".join([f" [Source {i+1}]: {chunk}" for i, chunk in enumerate(chunks)])
    prompt = f"""
User Query: "{state['user_query']}"

Here is the Retrieved Context from the paper:
===
{formatted_context}
===

Task: Write a comprehensive answer to the user query based on the context above.
    """

    sub_inputs = {"messages": [HumanMessage(content=prompt)]}
    sub_result = answer_gen_graph.invoke(sub_inputs)

    return {
        "answer_logs": sub_result["messages"],
        "process_status": "success"
    }

# +
def call_evaluate_agent(state: MainState):
    print("\n [Main] EvaluateAgent (Sub-Graph) 호출 ...")

    user_query = state["user_query"]

    if not state.get("answer_logs"):
        return {
            "eval_logs": [AIMessage(content="No answer to evaluate.")],
            "process_status": "eval_fail"
        }
    final_answer = state["answer_logs"][-1].content

    if not state.get("extract_logs"):
        context_text = "No context found."
    else:
        last_extract_msg = state["extract_logs"][-1]
        parsed_result = clean_and_parse_json(last_extract_msg.content)
        chunks = parsed_result.get("chunks", []) if parsed_result else []
        context_text = "\n\n".join(chunks)

    input_prompt = f"""
[User Query]:
{user_query}

[Retrieved Context]:
{context_text}

[Generated Answer]:
{final_answer}

Evaluate now based on the rubric. Output strict JSON.
"""

    sub_inputs = {"messages": [HumanMessage(content=input_prompt)]}
    sub_result = evaluate_graph.invoke(sub_inputs)

    return {
        "eval_logs": sub_result["messages"],
        "process_status": "audit_complete"
    }

def check_build_status(state: MainState):
    if not state.get("build_logs"):
        return END

    last_msg = state["build_logs"][-1]
    parsed = clean_and_parse_json(last_msg.content)

    if parsed and parsed.get("status") == "success":
        return "continue"
    return END

def check_extract_status(state: MainState):
    if not state.get("extract_logs"):
        return END

    last_msg = state["extract_logs"][-1]
    parsed = clean_and_parse_json(last_msg.content)

    if parsed and parsed.get("status") == "success":
        return "continue"
    return END

이제 정의된 세 함수를 전체 그래프의 노드로 등록하고, 각 노드별 연결점을 지정합니다.

In [31]:
super_workflow = StateGraph(MainState)

super_workflow.add_node("info_build_agent_workflow", call_info_builder)
super_workflow.add_node("info_extract_agent_workflow", call_info_extractor)
super_workflow.add_node("answer_gen_agent_workflow", call_answer_gen)
super_workflow.add_node("evaluate_agent_workflow", call_evaluate_agent) # +

super_workflow.set_entry_point("info_build_agent_workflow")

super_workflow.add_conditional_edges(
    "info_build_agent_workflow",
    check_build_status,
    {
        "continue": "info_extract_agent_workflow",
        END: END
    }
)

super_workflow.add_conditional_edges(
    "info_extract_agent_workflow",
    check_extract_status,
    {
        "continue": "answer_gen_agent_workflow",
        END: END
    }
)

# - +
super_workflow.add_edge("answer_gen_agent_workflow", "evaluate_agent_workflow")
super_workflow.add_edge("evaluate_agent_workflow", END)
super_graph = super_workflow.compile()

In [32]:
# !pip install nest_asyncio
# !pip install pyppeteer

# import nest_asyncio
from IPython.display import Image, display
from langchain_core.runnables.graph import MermaidDrawMethod

# nest_asyncio.apply()

# 그래프를 Mermaid PNG로 변환하여 출력
display(Image(super_graph.get_graph().draw_mermaid_png()))
# display(Image(super_graph.get_graph().draw_mermaid_png(draw_method=MermaidDrawMethod.PYPPETEER)))

ValueError: Failed to reach https://mermaid.ink API while trying to render your graph. Status code: 400.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

### LangGraph 그래프 실행

`super_graph.stream(inputs)`을 통해 비동기식 노드 실행 결과를 실시간으로 스트리밍받아 처리합니다. `graph.stream`은 LangGraph의 핵심 메서드로, 입력(`inputs`)을 기반으로 그래프 내 노드들을 순차/병렬로 실행하며, 각 노드의 완료 시점마다 이벤트를 방출합니다.

이벤트에는 노드 이름(`key`)과 해당 노드의 출력(`value`)이 포함되며, `build_logs`, `extract_logs`, `answer_logs` 등 작업 단계별 로그를 구조화하여 제공합니다. 특히, 로그의 마지막 항목(`[-1]`)만 추출해 간략히 출력함으로써 진행 상황 모니터링과 디버깅을 직관적으로 지원합니다. 최종 답변은 `answer_logs`에서 추출되며, 워크플로우 종료 시 `"🏁 Workflow Completed"`로 프로세스 완료를 명시적으로 알립니다. 이 방식은 복잡한 파이프라인에서 중간 결과 추적과 오류 진단을 용이하게 합니다.

In [36]:
from IPython.display import Image, display

query = "Llama 3 논문 처리해줘"
inputs = {"user_query": query, "process_status": "start"}

print(f"🚀 워크플로우 시작: '{query}'")

for event in super_graph.stream(inputs):
    print(event)
    for key, value in event.items():
        if "build_logs" in value:
            print(f"✅ [Builder]: {value['build_logs'][-1].content[:50]}...")
        elif "extract_logs" in value:
            print(f"✅ [Extractor]: {value['extract_logs'][-1].content[:50]}...")
        elif "answer_logs" in value:
            print(f"\n📝 [Generated Answer]:\n{value['answer_logs'][-1].content}\n")
        elif "eval_logs" in value:
            print(f"🕵️ [Auditor Evaluation]:\n{value['eval_logs'][-1].content}")

print("\n🏁 Workflow Completed.")

🚀 워크플로우 시작: 'Llama 3 논문 처리해줘'

 [Main] InfoBuildAgent 호출 ...
{'info_build_agent_workflow': {'build_logs': [HumanMessage(content='Llama 3 논문 처리해줘', additional_kwargs={}, response_metadata={}, id='d70b5bbb-8bab-4b2a-8094-45780256a0f8'), AIMessage(content='[Essential to identify the arXiv ID since none was provided. The search query directly targets the paper requested ("Llama 3") on arXiv to retrieve its ID.]  \n\n**Next steps after receiving the arXiv ID:**  \n1. Check if the paper is cached using `check_vector_db_cache(arxiv_id)`.  \n2. If not cached, download the PDF and parse/store it.  \n\nThe workflow will proceed strictly in the defined order. Final output will be the arXiv ID and status in JSON format.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 364, 'prompt_tokens': 1112, 'total_tokens': 1476, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 223, 'rejected_prediction_tokens': 